In [5]:
import pandas as pd
import numpy as np 
import sklearn
from scipy.stats import norm
import time
import os

import xgboost

from sklearn.model_selection import train_test_split, TimeSeriesSplit, RandomizedSearchCV,GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 200)


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Setting output dictionary
output_dir = '/home/dante/SpatialData/spatial_project/data/output/'

In [10]:
# Loading final data
df = pd.read_csv('/content/drive/MyDrive/spatial_data/final_project/master_data.csv').iloc[:,0:-1]
df.rename(columns = {'newcases_tplus1' : 'newcases_tplus2', 'newcases_tplus2' : 'newcases_tplus3', 'newcases_tplus4' : 'newcases_tplus5', 'newcases' : 'newcases_tplus1'},inplace=True)

In [11]:
df.head()

,province,Y-W,newcases_tplus1,newcases_tminus1,newcases_tminus2,newcases_tminus3,newcases_tminus4,newcases_tplus2,newcases_tplus3,newcases_tplus5,...,workplaces_tminus4,residential_tminus1,residential_tminus2,residential_tminus3,residential_tminus4,2019employment_rate,2019intermunicipal_migration_rate,2019annual_contrib_margin,2019share_age_64,2019number_workplaces
0,South Karelia,2020-01,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,67.4,-488,387.3,27.4,47995
1,Southern Ostrobothnia,2020-01,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,73.8,-935,123.6,25.4,77419
2,Southern Savonia,2020-01,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,68.9,-1228,143.5,30.9,50145
3,Kainuu,2020-01,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,68.3,-462,857.2,28.5,27218
4,Tavastia Proper,2020-01,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,73.1,-69,197.5,25.3,64199


In [ ]:
df.columns

Index(['province', 'Y-W', 'newcases', 'newcases_tminus1', 'newcases_tminus2',
       'newcases_tminus3', 'newcases_tminus4', 'newcases_tplus1',
       'newcases_tplus2', 'newcases_tplus4', 'spc_tminus1', 'spc_tminus2',
       'spc_tminus3', 'spc_tminus4', 'temp_min_tminus1', 'temp_min_tminus2',
       'temp_min_tminus3', 'temp_min_tminus4', 'temp_max_tminus1',
       'temp_max_tminus2', 'temp_max_tminus3', 'temp_max_tminus4',
       'retail_tminus1', 'retail_tminus2', 'retail_tminus3', 'retail_tminus4',
       'grocery_tminus1', 'grocery_tminus2', 'grocery_tminus3',
       'grocery_tminus4', 'parks_tminus1', 'parks_tminus2', 'parks_tminus3',
       'parks_tminus4', 'transit_tminus1', 'transit_tminus2',
       'transit_tminus3', 'transit_tminus4', 'workplaces_tminus1',
       'workplaces_tminus2', 'workplaces_tminus3', 'workplaces_tminus4',
       'residential_tminus1', 'residential_tminus2', 'residential_tminus3',
       'residential_tminus4', '2019employment_rate',
       '2019intermu

In [12]:
def choosehorizon(dataframe,pred : int,lags):
    """
    This function takes in the master dataframe and spits out the lags that you want to include,
    and then sorts the data so that it's easier to look at.
    """
    if pred not in [1,2,4]:
        print(f"Sorry, but the only available prediction horizons are 1,2 and 4 weeks ahead!")
        return None
    elif (max(lags) > 4) or (min(lags) < 1):
        if len(lags) == 0:
            print("You have specified zero lags! Nonsense!")
        else: print("Your lags are out of bounds...")
    
    lagcols = [col for col in list(dataframe.columns) if 'tminus' in col]
    
    lagschosen = []
    for lag in lags:
        lagschosen += [col for col in lagcols if str(lag) in col]
        
    lagschosen.sort()
    sociocols = [col for col in list(dataframe.columns) if '2019' in col]
    indices = ['Y-W','province']
    y = f"newcases_tplus{pred}"
    
    allcols = indices + [y] + lagschosen + sociocols
    
    # Given lags, take out data that contains NaNs due to data unavailability
    # First week when we have data is 2020-10, so add up weeks according to lags..
    max_lag = max(lags)
    # first_week = f"2020-{10+max_lag}"
    first_week = '2021-01'
    last_week = '2021-52'
    
    # Given that we are predicting into the future, we need to take off as many weeks as we
    # are predicting into the future for, because otherwise we have NaNs..
    last_week = f"2022-{10-pred}"
    
    # We also need to take care of the fact that we only have Google Mobility data up to 2021-52
    last_week_google = f"2021-52"
    
    last_effective_week = [last_week,last_week_google]
    last_effective_week.sort()
    dataframe = dataframe[(dataframe['Y-W'] >= first_week) & (dataframe['Y-W'] <= last_effective_week[0])]
    dataframe.reset_index(drop=True,inplace=True)
    return dataframe[allcols],y

In [13]:
data,depvarname = choosehorizon(df,1,[1,2,3,4])

In [17]:
google_data_list = [
"retail",
"grocery",
"parks",
"transit",
"workplaces",
"residential"
]

google_datas = []
for google_data in google_data_list:
  google_datas += [col for col in data.columns if google_data in col]

In [18]:
google_datas

['retail_tminus1',
 'retail_tminus2',
 'retail_tminus3',
 'retail_tminus4',
 'grocery_tminus1',
 'grocery_tminus2',
 'grocery_tminus3',
 'grocery_tminus4',
 'parks_tminus1',
 'parks_tminus2',
 'parks_tminus3',
 'parks_tminus4',
 'transit_tminus1',
 'transit_tminus2',
 'transit_tminus3',
 'transit_tminus4',
 'workplaces_tminus1',
 'workplaces_tminus2',
 'workplaces_tminus3',
 'workplaces_tminus4',
 '2019number_workplaces',
 'residential_tminus1',
 'residential_tminus2',
 'residential_tminus3',
 'residential_tminus4']

In [20]:
data.drop(columns = google_datas,inplace=True)

In [21]:
data.head()

,Y-W,province,newcases_tplus1,newcases_tminus1,newcases_tminus2,newcases_tminus3,newcases_tminus4,spc_tminus1,spc_tminus2,spc_tminus3,...,temp_max_tminus3,temp_max_tminus4,temp_min_tminus1,temp_min_tminus2,temp_min_tminus3,temp_min_tminus4,2019employment_rate,2019intermunicipal_migration_rate,2019annual_contrib_margin,2019share_age_64
0,2021-01,South Karelia,0.743388,0.915088,0.581037,1.236188,0.488772,1.059205,0.813258,1.179916,...,1.414286,-2.014286,-3.385714,-2.557143,-1.614286,-5.328571,67.4,-488,387.3,27.4
1,2021-01,Southern Ostrobothnia,0.000000,0.000000,0.231281,0.231281,0.919979,0.987278,0.742646,1.177277,...,2.600000,-1.385714,-4.342857,-2.928571,-0.985714,-4.100000,73.8,-935,123.6,25.4
2,2021-01,Southern Savonia,0.000000,0.319750,0.471786,1.408454,1.479570,1.043021,0.784992,1.085412,...,0.771429,-1.171429,-7.200000,-4.471429,-4.314286,-4.442857,68.9,-1228,143.5,30.9
3,2021-01,Kainuu,0.983755,0.000000,0.000000,0.000000,0.000000,1.015423,0.718770,0.996697,...,1.714286,-1.885714,-1.642857,-1.842857,-0.685714,-5.242857,68.3,-462,857.2,28.5
4,2021-01,Tavastia Proper,1.323183,0.902441,0.948898,1.224836,1.398341,1.110361,0.976796,1.292484,...,-3.042857,-2.357143,-10.142857,-8.771429,-8.714286,-6.157143,73.1,-69,197.5,25.3


In [14]:
data.shape[0]/18

52.0

## Training setup

In [ ]:
training_size = 30 # week
testing_size = 1 # week
num_counties = len(data.province.value_counts().index)
time_steps = 22

## Model tracking

In [ ]:
train_r2_xgb = dict()
train_rmse_xgb = dict()
train_mae_xgb = dict()
test_rmse_xgb = dict()
test_mae_xgb = dict()
tuned_params_xgb = dict()

## Model grid setup

In [ ]:
# Setting Hyperparameters. Please refer to the SI for more information
xgb_params = dict(learning_rate=np.arange(0.05,0.3,0.05),
                    #  n_estimators=np.arange(150,400,100), 
                     gamma = np.arange(1,3,1),
                     max_depth=[int(i) for i in np.arange(1,10,1)]) 

In [ ]:
xgb_params

{'gamma': array([1, 2]),
 'learning_rate': array([0.05, 0.1 , 0.15, 0.2 , 0.25]),
 'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9]}

## Model training

In [ ]:
for i in range(time_steps):
    
    training_df = data.iloc[:(i+training_size)*num_counties,:]
    testing_df = data.iloc[(i+training_size)*num_counties:(i+training_size+testing_size)*num_counties,:]
    
#     start_time = time.time()

    # in the 2-week prediction model, the target variable is LOG_DELTA_INC_RATE_T_14
    X_train = training_df.iloc[:,3:]
    y_train = training_df[depvarname]
    X_test = testing_df.iloc[:,3:]
    y_test = testing_df[depvarname]
    
    print(X_train.shape)
    print(y_train.shape)
    
    print(X_test.shape)
    print(y_test.shape)
    
    #scaling X
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    
    #inititalization
    xgb_model = xgboost.XGBRegressor(seed=42, verbosity=0)
    
    #cross validation
    xgb_cv = GridSearchCV(xgb_model, xgb_params, 
                                    scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=3)
    
    xgb_optimized = xgb_cv.fit(X_train, y_train)
    best_xgb = xgb_optimized.best_estimator_
    tuned_params_xgb['whole', i] = xgb_optimized.best_params_
    
    # model evaluation for training set
    r2_train_xgb = round(best_xgb.score(X_train, y_train),2)
    train_r2_xgb['whole', i] = r2_train_xgb
    
    y_train_predicted_xgb = best_xgb.predict(X_train)
    rmse_train_xgb = (np.sqrt(mean_squared_error(y_train, y_train_predicted_xgb)))
    train_rmse_xgb['whole', i] = rmse_train_xgbpd.get_dummies(moviegenres.apply(pd.Series).stack())
    train_mae_xgb['whole', i] =  mean_absolute_error(y_train, y_train_predicted_xgb)


    # model evaluation for test set
    y_test_predicted_xgb = best_xgb.predict(X_test)
    rmse_test_xgb = (np.sqrt(mean_squared_error(y_test, y_test_predicted_xgb)))
    test_rmse_xgb['whole', i] = rmse_test_xgb
    test_mae_xgb['whole', i] = mean_absolute_error(y_test, y_test_predicted_xgb)
        
    print(f"Finished {i}th round...")


In [ ]:
tuned_params_xgb

In [ ]:
print(np.exp(y_test_predicted_xgb - 1))
print(np.exp(y_test - 1))